# Robyn MMM 

## 1) Importare pacchetti, scegliere la directory del json del modello e caricare il dataset

### Importare pacchetti 

In [ ]:
library(readxl)
library(reshape2)
library(ggplot2)
library(googlesheets4) 
library(reshape2)
library(Robyn)
library(lubridate) 
library(png) # facoltativo
library(grid) # facoltativo
library(magick) # facoltativo

### Scegliere la directory per il file json finale 

In [ ]:
create_files <- TRUE
robyn_directory <- "~/Desktop"
file_path <- ".xlsx"  

### Valutare la classe delle variabili DATE e Week

In [ ]:
class(df$Week)

In [ ]:
class(df$DATE)

La variabile week è numeirca, mentre la variabile DATE è considerata come una stringa. Dunque, bisogna procedere a trasformala in formato as.date

In [ ]:
df$DATE <- dmy(df$DATE)

In [ ]:
class(df$DATE)

## 2) Gestione dei valori mancanti (NA)

### Righe con almeno un NA

In [ ]:
num_rows_with_na <- sum(apply(df, 1, function(row) any(is.na(row))))

In [ ]:
print(paste("Numero di righe con almeno un NA:", num_rows_with_na))

### Righe con più di un NA

In [ ]:
num_rows_with_multiple_nas <- sum(apply(df, 1, function(row) sum(is.na(row)) > 1))

In [ ]:
print(paste("Numero di righe con più di un NA:", num_rows_with_multiple_nas))

### Controllare il numero di NA per colonna 

In [ ]:
colSums(is.na(df))

### Visualizzare righe con NA

In [ ]:
rows_with_na <- !complete.cases(df)

In [ ]:
df_with_na <- df[rows_with_na, ]
print("Righe con almeno un NA:")
print(df_with_na)

In questo caso specifico, molti NA si trovano nelle settimane successive al 19/08/2024 perché mancano i dati forniti dal cliente. Dunque, si può procedere con l'eliminazione di queste righe dal momento che non si trovano nella parte centrale del dataset, ma nella parte finale (Si può fare questa operazione anche quando i dati si trovano nella parte iniziale).

In [ ]:
df <- df[1:236, ]

In [ ]:
tail(df)

### Rivedere i dati mancanti una volta effettuata questa operazione

In [ ]:
num_rows_with_na <- sum(apply(df, 1, function(row) any(is.na(row))))

In [ ]:
print(num_rows_with_na)

Valutare dove si trovano questi NA riproponendo il codice che valuta gli NA per colonna

In [ ]:
colSums(is.na(df))

I dati mancanti si trovano nelle colonne: compleanno_yeppon; compleanno_yeppon_weighted; black_friday; black_friday_weighted; error_404; marzo; marzo_weighted. Tutte colonne che dovrebbero avere degli 0, ragion per cui procediamo con la sostituzione degli NA con 0

In [ ]:
df[is.na(df)] <- 0

Verifico

In [ ]:
num_rows_with_na <- sum(apply(df, 1, function(row) any(is.na(row))))

In [ ]:
print(num_rows_with_na)

Tutti gli NA presenti nel dataset sono stati gestiti 

## 3) Matrice di correlazione per valutare le correlazioni tra le varianbili e verificare la multicollinearità

### Creare la matrice di correlazione e plottare heatmap

In [ ]:
matrix_data <- df[, c("facebook_cost", "search_cost", "shopping_cost", "bingS_cost", "trova_cost","display_cost","discovery_cost","youtube_cost","bingP_cost","rtg_cost","rtbhouse_cost","criteo_cost","tv_cost","banner_cost","radio_cost","ctv_cost")]
correlation_matrix <- cor(matrix_data)


In [ ]:
melted_cor_matrix <- melt(correlation_matrix)
ggplot(data = melted_cor_matrix, aes(x = Var1, y = Var2, fill = value)) +
  geom_tile() +
  scale_fill_gradient2(low = "blue", high = "red", mid = "white", 
                       midpoint = 0, limit = c(-1, 1), space = "Lab", 
                       name = "Correlazione") +
  theme_minimal() + 
  theme(axis.text.x = element_text(angle = 45, vjust = 1, hjust = 1)) +
  coord_fixed()

### Trovare con codice le variabili che hanno una correlazione >70%

In [ ]:
threshold <- 0.7

In [ ]:
high_cor_pairs <- which(abs(correlation_matrix) > threshold & abs(correlation_matrix) < 1, arr.ind = TRUE)

In [ ]:
high_cor_pairs_df <- data.frame(
  Var1 = rownames(correlation_matrix)[high_cor_pairs[, 1]],
  Var2 = colnames(correlation_matrix)[high_cor_pairs[, 2]],
  Correlation = correlation_matrix[high_cor_pairs]
)

In [ ]:
high_cor_pairs_df <- high_cor_pairs_df[!duplicated(t(apply(high_cor_pairs_df, 1, sort))), ]

In [ ]:
print(high_cor_pairs_df)

In questo caso notiamo che criiteo e rtg hanno una correlazione di 86%; radio e banner 99%; ctv e banner 99%. Dunque, è necessario tenere conto di queste correlazioni forti nella scelta delle variabili del modello, magari eliminando alcune di queste variabili

## 4) Creazione del Modello di MMM

### InputCollect

In [ ]:
InputCollect <- robyn_inputs(
  dt_input = df,          # nome del dataset in uso
  dt_holidays = dt_prophet_holidays,
  date_var = "DATE",      # variabile data
  dep_var = "revenue",    # variabile dipendente 
  dep_var_type = "revenue",
  prophet_vars = c("trend", "season", "holiday"),    # Lasciare trend, stagionalità e vacanze così
  prophet_country = "IT",       # festività della nazione in esame
  paid_media_spends = c("facebook_cost", "search_cost", "shopping_cost","bingS_cost","display_cost","rtbhouse_cost","criteo_cost","tv_cost","banner_cost","radio_cost","rtg_cost","ctv_cost","youtube_cost"), # costi dei canali media scelti per il modello
  paid_media_vars = c("facebook_impression", "search_impression", "shopping_impression", "bingS_click","display_impression","rtbhouse_impression","criteo_impression","tv_grp","banner_impression","radio_impression","rtg_impression","ctv_impression","youtube_impression"), # impression dei canali scelti 
  window_start = "2020-05-11", 
  window_end = "2024-05-13",
  adstock = "geometric"   # Lasciare adstock geometrico
  )

In [ ]:
print(InputCollect)

In [ ]:
hyper_names(adstock = InputCollect$adstock, all_media = InputCollect$all_media)

### Creazione iperparametri

In [ ]:
hyperparameters <- list(
 banner_cost_alphas = c(0.5, 3),      
 banner_cost_gammas = c(0.3, 1),      
 banner_cost_thetas = c(0.05, 0.10),  
 bingS_cost_alphas = c(0.5, 3),
 bingS_cost_gammas = c(0.3, 1),
 bingS_cost_thetas = c(0.05, 0.10),
 criteo_cost_alphas = c(0.5, 3),
 criteo_cost_gammas = c(0.3, 1),
 criteo_cost_thetas = c(0.05, 0.10),   # valori maggiori di alpha rendono la curva più a S, mentre valori minori la rendono più a C.
 display_cost_alphas = c(0.5, 3),
 display_cost_gammas = c(0.3, 1),
 display_cost_thetas = c(0.05, 0.10),
 facebook_cost_alphas = c(0.5, 3),
 facebook_cost_gammas = c(0.3, 1),
 facebook_cost_thetas = c(0.15, 0.3),  # valori più alti di gamma spostano l'inflessione verso date successive, ritardando la saturazione dell'effetto.
 radio_cost_alphas = c(0.5, 3),
 radio_cost_gammas = c(0.3, 1),
 radio_cost_thetas = c(0.10, 0.20),
 rtbhouse_cost_alphas = c(0.5, 3),
 rtbhouse_cost_gammas = c(0.3, 1),
 rtbhouse_cost_thetas = c(0.1, 0.10),  # valori più alti di theta comportano una decadenza più lenta dell'effetto pubblicitario nel tempo.
 rtg_cost_alphas = c(0.5, 3),
 rtg_cost_gammas = c(0.3, 1),
 rtg_cost_thetas = c(0.05, 0.10),
 search_cost_alphas = c(0.5, 3),
 search_cost_gammas = c(0.3, 1),
 search_cost_thetas = c(0.1, 0.12),
 shopping_cost_alphas = c(0.5, 3),
 shopping_cost_gammas = c(0.3, 1),
 shopping_cost_thetas = c(0.1, 0.13),  
 youtube_cost_gammas = c(0.3, 1),
 tv_cost_alphas = c(0.5, 3),
 tv_cost_gammas = c(0.3, 1),
 tv_cost_thetas = c(0.10, 0.20),
 youtube_cost_alphas = c(0.5, 3),       
 youtube_cost_thetas = c(0.05, 0.10),
 ctv_cost_alphas = c(0.5, 3),
 ctv_cost_gammas = c(0.3, 1),
 ctv_cost_thetas = c(0.05, 0.10),
 train_size = c(0.5, 0.8)
 )


In [ ]:
InputCollect <- robyn_inputs(InputCollect = InputCollect, hyperparameters = hyperparameters)

In [ ]:
print(InputCollect)

### Creazione modelli 

In [ ]:
OutputModels <- robyn_run(
 InputCollect = InputCollect, 
 iterations = 2000, 
 trials = 5, 
 ts_validation = F, 
 add_penalty_factor = FALSE 
) 

In [ ]:
OutputCollect <- robyn_outputs(
  InputCollect, OutputModels,
  pareto_fronts = "auto", 
  csv_out = "pareto", 
  clusters = TRUE, 
  plot_pareto = create_files 
)


Cercare i modelli in questa cartella 'Robyn_202409172304_init' e scegliere il modello migliore 

Le prossime 3 righe di codice non sono necessarie, servono solo a visualizzare le immagini nel notebook

### Scelta modello ed esportazione 

In [ ]:
select_model <- "1_285_5"

In [ ]:
ExportedModel <- robyn_write(InputCollect, OutputCollect, select_model, export = create_files)

In [ ]:
print(ExportedModel)

## 5) Allocazione del budget 

In [ ]:
AllocatorCollect <- robyn_allocator(
  InputCollect = InputCollect,
  OutputCollect = OutputCollect,
  select_model = select_model,
  date_range = "all", # Default to "all", altrimenti si può inserire un range di date entro il quali effettuare l'allocazione
  total_budget = NULL, # When NULL, default is total spend in date_range, altrimenti si può inserire un budget manualmente
  channel_constr_low = 0.7, # limite inferiore
  channel_constr_up = c(1.5, 1.6, 1.7, 1.2, 1.1,1.1,1.9,1.5,1.5,1.5,1.5,1.5,1.5), # limite superiore
  scenario = "max_response",
  export = create_files
)

Le prossime 3 righe di codice non sono necessarie, servono solo a visualizzare le immagini nel notebook

In [ ]:
image_path_2 <-'/Users/digitalangels/Robyn_202409172304_init/1_285_5_reallocated_best_roas.png'

In [ ]:
img_2 <- readPNG(image_path_2)

In [ ]:
grid.raster(img_2)

## 5) Refresh (Es. Q4 2024)

### Rifare la stessa procedura di caricamento del dataset e gestione dati mancanti 

### Caricamento del file json del modello da cui effettuare refresh 

In questo caso viene fatto un refresh di un refresh: viene caricato il file json del refresh Q3 2024

In [ ]:
json_file <- "~/Desktop/RobynModel-1_136_7.json"

### Effettuare il refresh 

In [ ]:
RobynRefresh <- robyn_refresh(
  json_file = json_file,
  dt_input = df,
  dt_holidays = dt_prophet_holidays,
  window_start = "27-05-2024", # inserire la data di partenza da cui effettuare refresh
  refresh_steps = 13, # numero di settimane aggiunte 
  refresh_iters = 1000, 
  refresh_trials = 1
)

Cercare modello in questa cartella 'Robyn_202409172344_rf2'

Le prossime 3 righe di codice non sono necessarie, servono solo a visualizzare le immagini nel notebook

### Creazione InputCollect, OutputCollect e selezione modello ottenuto

In [ ]:
InputCollectX <- RobynRefresh$listRefresh1$InputCollect
OutputCollectX <- RobynRefresh$listRefresh1$OutputCollect
select_modelX <- RobynRefresh$listRefresh1$OutputCollect$selectID

### Allocazione budget del refresh

In [ ]:
AllocatorCollect <- robyn_allocator(
  InputCollect = InputCollectX,
  OutputCollect = OutputCollectX,
  select_model = select_modelX,
  date_range = c("2023-07-03", "2023-09-25"), # scegliere il range temporale dell'anno precedente che corrisponde al periodo dell'anno corrente che si vuole simulare 
  total_budget = 216660, # Inserire il budget disponibile
  channel_constr_low = 0.7,
  channel_constr_up = c(1.5, 1.5, 1.2, 1.2, 1.5,1.5,1.5,1.5,1.7,1.1,1.1),
  scenario = "max_response",
  export = create_files
)

Cercare in questa cartella 'Robyn_202409172344_rf2'

Le prossime 3 righe di codice non sono necessarie, servono solo a visualizzare le immagini nel notebook